In [92]:
import os
import arff
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import mlflow
import numpy as np



In [93]:
RANDOM_SEED = 10


## Data Exploration

In [94]:
data_dir = './data'

#   Features
data_freq = arff.load(os.path.join(data_dir, 'freMTPL2freq.arff'))

df_freq = pd.DataFrame(data_freq , columns=["IDpol", "ClaimNb", "Exposure", "Area", "VehPower",
"VehAge", "DrivAge", "BonusMalus", "VehBrand", "VehGas", "Density", "Region"] )


# Labels

data_sev = arff.load(os.path.join(data_dir, 'freMTPL2sev.arff'))

df_sev = pd.DataFrame(data_sev, columns=["IDpol", "ClaimAmount"])

In [95]:
display(df_freq.head())

print("Dataset size: ", len(df_freq))

print("IDpol has only unique values? ", (df_freq['IDpol'].nunique() == len(df_freq['IDpol'])))

display('Description of dataset: ', df_freq.describe(include='all'))



,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,3.0,1.0,0.77,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,5.0,1.0,0.75,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,10.0,1.0,0.09,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,11.0,1.0,0.84,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'


Dataset size:  678013
IDpol has only unique values?  True


'Description of dataset: '

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
count,6.780130e+05,678013.000000,678013.000000,678013,678013.000000,678013.000000,678013.000000,678013.000000,678013,678013,678013.000000,678013
unique,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,11,2,NaN,22
top,NaN,NaN,NaN,'C',NaN,NaN,NaN,NaN,'B12',Regular,NaN,'R24'
freq,NaN,NaN,NaN,191880,NaN,NaN,NaN,NaN,166024,345877,NaN,160601
mean,2.621857e+06,0.053247,0.528750,NaN,6.454631,7.044265,45.499122,59.761502,NaN,NaN,1792.422405,NaN
std,1.641783e+06,0.240117,0.364442,NaN,2.050906,5.666232,14.137444,15.636658,NaN,NaN,3958.646564,NaN
min,1.000000e+00,0.000000,0.002732,NaN,4.000000,0.000000,18.000000,50.000000,NaN,NaN,1.000000,NaN
25%,1.157951e+06,0.000000,0.180000,NaN,5.000000,2.000000,34.000000,50.000000,NaN,NaN,92.000000,NaN
50%,2.272152e+06,0.000000,0.490000,NaN,6.000000,6.000000,44.000000,50.000000,NaN,NaN,393.000000,NaN
75%,4.046274e+06,0.000000,0.990000,NaN,7.000000,11.000000,55.000000,64.000000,NaN,NaN,1658.000000,NaN


In [96]:
display(df_sev.head())

print("Dataset size: ", len(df_sev))

print("Percentage of IDpol-contracts with only one ClaimAmount: ", 
      round(df_sev['IDpol'].nunique()/len(df_sev['IDpol']), 4))

display('Description of dataset: ', df_sev.describe(include='all'))


print('Data types in ClaimAmount column:', df_sev["ClaimAmount"].apply(type).unique(), '\n')



,IDpol,ClaimAmount
0,1552.0,995.20
1,1010996.0,1128.12
2,4024277.0,1851.11
3,4007252.0,1204.00
4,4046424.0,1204.00


Dataset size:  26639
Percentage of IDpol-contracts with only one ClaimAmount:  0.9366


'Description of dataset: '

,IDpol,ClaimAmount
count,2.663900e+04,2.663900e+04
mean,2.279864e+06,2.278536e+03
std,1.577202e+06,2.929748e+04
min,1.390000e+02,1.000000e+00
25%,1.087642e+06,6.868100e+02
50%,2.137413e+06,1.172000e+03
75%,3.180162e+06,1.228080e+03
max,6.113971e+06,4.075401e+06


Data types in ClaimAmount column: [<class 'float'>] 



## Data preprocessing

### Create Features and Labels Dataframe

In [97]:
#   Sum up ClaimAmount for each distinct IDpol (contract)

df_sev_sum = df_sev.groupby('IDpol').sum().reset_index()

display(df_sev_sum)

,IDpol,ClaimAmount
0,139.0,303.00
1,190.0,1981.84
2,414.0,1456.55
3,424.0,10834.00
4,463.0,3986.67
...,...,...
24945,6113521.0,1324.40
24946,6113793.0,1769.88
24947,6113817.0,1288.28
24948,6113834.0,12230.40


In [98]:
#   Left join features table with ClaimAmount-table (containing unique IDpol)

df_combined = df_freq.merge(df_sev_sum, on="IDpol", how='left')

#   Fill null values of ClaimAmount with 0.0 (no claim was made) 

df_combined["ClaimAmount"] = df_combined["ClaimAmount"].fillna(value=0.0)

display(df_combined)


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,1.0,1.0,0.10000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82',0.0
1,3.0,1.0,0.77000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82',0.0
2,5.0,1.0,0.75000,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22',0.0
3,10.0,1.0,0.09000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72',0.0
4,11.0,1.0,0.84000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72',0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,'E',4.0,0.0,54.0,50.0,'B12',Regular,3317.0,'R93',0.0
678009,6114327.0,0.0,0.00274,'E',4.0,0.0,41.0,95.0,'B12',Regular,9850.0,'R11',0.0
678010,6114328.0,0.0,0.00274,'D',6.0,2.0,45.0,50.0,'B12',Diesel,1323.0,'R82',0.0
678011,6114329.0,0.0,0.00274,'B',4.0,0.0,60.0,50.0,'B12',Regular,95.0,'R26',0.0


In [99]:
#   Check if num of IDpols is correct

if len(df_combined) != len(df_freq):
    raise Exception("Wrong dimensions for df_combined")

In [100]:
#   Randomly drop certain amount of No-Claim cases 
#   so that balance between Claim <-> No-Claim cases is better

#   Given ~25,000 Claim-Cases, I choose to keep ~25,000 of the No-Claim cases -> drop ~625,000 No-Claim Cases

# get indices of No-Claim cases
no_claim_cases_indices = df_combined[df_combined["ClaimAmount"] == 0.0].index

#   Important to put here (cell re-run...)
np.random.seed(RANDOM_SEED)

indices_for_dropping = np.random.choice(no_claim_cases_indices, 
                                        size=625000, 
                                        replace=False)


df_combined_reduced = df_combined.drop(index=indices_for_dropping)

display(df_combined_reduced)


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
10,25.0,1.0,0.750000,'B',7.0,0.0,41.0,50.0,'B12',Diesel,60.0,'R52',0.0
15,36.0,1.0,0.340000,'F',9.0,0.0,44.0,76.0,'B12',Regular,27000.0,'R11',0.0
19,45.0,1.0,0.100000,'A',6.0,2.0,55.0,50.0,'B12',Regular,37.0,'R94',0.0
21,49.0,2.0,0.810000,'E',7.0,0.0,73.0,50.0,'B12',Regular,3317.0,'R93',0.0
24,53.0,1.0,0.550000,'D',5.0,0.0,33.0,100.0,'B12',Regular,1746.0,'R11',0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677922,6114240.0,0.0,0.030000,'F',4.0,0.0,31.0,100.0,'B12',Regular,22669.0,'R11',0.0
677957,6114275.0,0.0,0.010000,'F',6.0,10.0,31.0,54.0,'B2',Diesel,13520.0,'R11',0.0
677959,6114277.0,0.0,0.010000,'D',11.0,1.0,53.0,50.0,'B12',Diesel,1955.0,'R94',0.0
677995,6114313.0,0.0,0.005479,'E',4.0,0.0,44.0,50.0,'B12',Regular,3457.0,'R82',0.0


In [101]:
print(len(df_combined_reduced[df_combined_reduced["ClaimAmount"] > 0.0]))

24944


In [102]:
#   Create features dataset

X = df_combined_reduced.drop(columns=["IDpol", "ClaimNb", "Exposure", "ClaimAmount"]).reset_index(drop=True)

print("Features:")
display(X)

Features:


,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,'B',7.0,0.0,41.0,50.0,'B12',Diesel,60.0,'R52'
1,'F',9.0,0.0,44.0,76.0,'B12',Regular,27000.0,'R11'
2,'A',6.0,2.0,55.0,50.0,'B12',Regular,37.0,'R94'
3,'E',7.0,0.0,73.0,50.0,'B12',Regular,3317.0,'R93'
4,'D',5.0,0.0,33.0,100.0,'B12',Regular,1746.0,'R11'
...,...,...,...,...,...,...,...,...,...
53008,'F',4.0,0.0,31.0,100.0,'B12',Regular,22669.0,'R11'
53009,'F',6.0,10.0,31.0,54.0,'B2',Diesel,13520.0,'R11'
53010,'D',11.0,1.0,53.0,50.0,'B12',Diesel,1955.0,'R94'
53011,'E',4.0,0.0,44.0,50.0,'B12',Regular,3457.0,'R82'


In [103]:
Y = df_combined_reduced[["ClaimAmount", "Exposure"]].reset_index(drop=True)

print("Labels: ")
display(Y)

Labels: 


,ClaimAmount,Exposure
0,0.0,0.750000
1,0.0,0.340000
2,0.0,0.100000
3,0.0,0.810000
4,0.0,0.550000
...,...,...
53008,0.0,0.030000
53009,0.0,0.010000
53010,0.0,0.010000
53011,0.0,0.005479


###  One-hot encode VehGas and Area (low cardinality, no natural ordering)

### Splitting data

In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED)


## Create Model

In [105]:
# Gradient-Boosted Decision Trees; GradientBoostingRegressor or RandomForestRegressor

